#### Unsupervised classification with K-means using GEE

In [29]:
# Import modules and initialize
import ee, json, folium
ee.Initialize()

In [122]:
# Open geoJSON file and store it in a new variable

with open("kings_creek.geojson") as file:
    kings_creek_boundary = json.load(file)
    

In [123]:
# Define GEE GEometry

kc_geometry = ee.Geometry(kings_creek_boundary['features'][0]['geometry'])

In [124]:
# Test GEE Geometry import printing the area in sq km2

kc_geometry.area().getInfo()/1000000


11.353858144557279

In [126]:
# Import source dataset to generate the clusters

weather = ee.Image("WORLDCLIM/V1/BIO").select(['bio01', 'bio12']).resample('bicubic')
soil = ee.Image("projects/soilgrids-isric/sand_mean").select('sand_0-5cm_mean').resample('bicubic')
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1").filterDate('2022-05-01', '2022-05-30').first().resample('bicubic')

dataset = weather.addBands(soil).addBands(ndvi)

In [127]:
# Sample points across the image in the defined region

sample_points = dataset.sample(region=kc_geometry, scale=20, numPixels=1000)

In [128]:
# Define number of cluster to classify

cluster_count = 5
classificator = ee.Clusterer.wekaKMeans(cluster_count).train(sample_points)

In [129]:
# Generate the clusters for the whole region

clusters = dataset.cluster(classificator).reduceToVectors(scale=20, geometry=kc_geometry).getInfo()

In [130]:
# clusters

In [131]:
# Create Folium Map
m = folium.Map(location=[39.07648, -96.592], zoom_start=12)

# Get Color for different polygons
def get_polygon_color(polygon):
    colors = ['#7b3294','#c2a5cf','#f7f7f7','#a6dba0','#008837']
    color = colors[polygon['properties']['label']]
    return {'fillColor': color, 'fillOpacity': 0.8}
    
    
cluster_layer=folium.GeoJson(clusters, name="Konza Praririe Clustered Regions", style_function = get_polygon_color)
cluster_layer.add_to(m)

m

In [100]:
# Save result to a geoJSON file

with open('output.json', 'w') as file:
    json.dump(clusters, file)